<a href="https://colab.research.google.com/github/alexisakov/RTPI/blob/master/Hard_numbers_Case_study_1_Label_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case 1. Product label classification: 'The Dumpling' 

Import Yandex MyStem for lemmatizatio + requests for talking to API + scikit-learn which will do most of the work.

In [ ]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

In [23]:
pip install pymystem3

In [24]:
from pymystem3 import Mystem

In [39]:
import requests
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [52]:
m = Mystem()

Set up connection to the Hard Numbers API - you will need your API Key - reach us to get one: https://t.me/xiskv.

In [26]:
base_url='http://rtpiapi.hrdn.io/'

token = 'YOUR API KEY GOES HERE'

request_headers = {'Authorization': f'Bearer {token}',
                    'Content-Type': 'application/json',
                    'Range-Unit': 'items'
                  }

Here we get all the text label data:

In [27]:
request_url = base_url + 'rtpi_product_name'

response = requests.get(request_url, headers = request_headers).json()

Overall there is about 153k labels

In [28]:
len(response)

153625

In [29]:
response[0]

{'contributor_id': 1,
 'product_name': 'Круассаны Olga Voloza с вареной сгущенкой 325г',
 'web_price_id': 133490}

Now let's make a call to both text lable table rtpi_product_name and the general dictionary of traked products rtpi_price_page so that we get those text labels that we have already matched with the Rosstat's codebook:

In [30]:
request_url = base_url + 'rtpi_product_name?select=*,rtpi_price_page(rosstat_id)&(rosstat_id.rosstat_id.not.is.null)'
responseXY = requests.get(request_url, headers = request_headers).json()
responseXY = [x for x in responseXY if (x['rtpi_price_page']['rosstat_id'])]

In [94]:
len(responseXY)

13127

For reference here is the Rosstat's current codebook: https://rosstat.gov.ru/storage/mediabank/j3LP7dsR/nabor_2020.xlsx

Okay, here is a bit of flattenting of our list of dicts - just to keep things tidy:

In [ ]:
[x.update(x['rtpi_price_page']) for x in responseXY];
[x.pop('rtpi_price_page',None) for x in responseXY];

 Now, let's use Y.MyStem to standardize/lemmatize product labels: 

In [ ]:
[x.update({'product_name': ''.join(m.lemmatize(x['product_name'])).rstrip()}) for x in responseXY];
[x.update({'product_name': ''.join(m.lemmatize(x['product_name'])).rstrip()}) for x in response];

In [38]:
responseXY[0]

{'contributor_id': 1,
 'product_name': 'сухой корм для собака Profine Light ягненок 15кг',
 'rosstat_id': 8311,
 'web_price_id': 148128}

Okay - now the most interesting part - let's fit the logistic regression to the text data - this will allow us to use machine learning to automate labelling - whoa! 


In [40]:
xydf = pd.DataFrame(responseXY)

In [41]:
xydf.head()

,web_price_id,product_name,contributor_id,rosstat_id
0,148128,сухой корм для собака Profine Light ягненок 15кг,1,8311
1,148455,сухой корм для собака Pro Plan Optiweight Smal...,1,8311
2,149127,томат маркет перекресток резаный 390г,1,1302
3,149296,горошек Bonduelle зеленый молодой 400г,1,1302
4,150154,кетчуп Calve баварский 350мл,1,1305


To keep things simple I choose to fit the model to classify just one type of product - dumplings, which have Rosstat's code of '106':

In [ ]:
targetProductCode = 106 

In [43]:
Y = xydf['rosstat_id'].apply(lambda x: 1 if x == targetProductCode else 0)

There is this number of dumplings which we have manually labeled at this point:

In [44]:
Y.sum()

105

Now vectorize our text labels:

In [45]:
tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.95)

In [46]:
tfidf_vectorizer.fit_transform(xydf['product_name'].values)

<13127x10157 sparse matrix of type '<class 'numpy.float64'>'
	with 84432 stored elements in Compressed Sparse Row format>

In [47]:
X = tfidf_vectorizer.transform(xydf['product_name'].values)

Fit the linear regression:

In [49]:
scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)

In [50]:
model=scikit_log_reg.fit(X,Y)

[LibLinear]

Extract class probabilities and get k observations with maximum probability of being a dumpling:

In [53]:
probs = model.predict_proba(X)

In [71]:
 k = 20

In [64]:
best_n = np.argsort(probs, axis=0)

In [75]:
xydf['product_name'].values[best_n[-k:,1]]

array(['пельмень сибирский коллекция домашние 700г',
       'пельмень сибирский коллекция русский 700г',
       'пельмень рублевкий сибирский 800г',
       'пельмень рублевский русский 800г',
       'пельмень мясной дворик халяль сибирский 800г',
       'пельмень дым домашние 800г',
       'пельмень мираторга из телятина 800г',
       'пельмень сибирский коллекция классический 700г',
       'пельмень стародворье сливочный 800г',
       'пельмень мираторга из мраморный говядина 800г',
       'пельмень мираторга деревенский 800г',
       'пельмень мираторга домашние из свинина и говядина 800г россия',
       'пельмень вкусвилл с говядина и зелень 800г',
       'пельмень мираторга домашние из свинина и говядина 800г',
       'пельмень мираторга нежный 800г', 'пельмень мираторга свиной 800г',
       'пельмень мираторга классический мини 800г',
       'пельмень мираторга мясной 800г',
       'пельмень вкусвилл с индейка 800г',
       'пельмень вкусвилл с говядина и свинина 800г'], dtype=obj

Good enough!

Now it is time to realy test our approach - and see how does it perform out of sample.

First, we filter the whole sample so that no training data slips into it:


In [85]:
trainid = [y['web_price_id'] for y in responseXY]

In [86]:
test = [x for x in response if x['web_price_id'] not in trainid]

In [88]:
testdf = pd.DataFrame(test)

Do the same routine: vectorize labels - evaluate probabilities - get top k observations with highets probabilities of being a dumpling:

In [89]:
testX = tfidf_vectorizer.transform(testdf['product_name'].values)

In [90]:
testprobs = model.predict_proba(testX)

In [91]:
test_best_n = np.argsort(testprobs, axis=0)

In [92]:
testdf['product_name'].values[test_best_n[-k:,1]]

array(['пельмень сибирский коллекция DамплS мясной 150г',
       'пельмень мой узол отборный 800г', 'пельмень классический 1кг',
       'пельмень крутогорский с масло 400г',
       'пельмень крутогорский классический 400г',
       'пельмень сочнов с отборный говядина 900г',
       'пельмень пельменный бутик Jimmy Щу из щука 400г',
       'пельмень равиолло равиоли 450г',
       'пельмень пельменный бутик говяджи & Kabban говядина и свинина 400г',
       'пельмень мираторга фермерский 800г',
       'пельмень крутогорский с масло 900г',
       'пельмень крутогорский классический 900г',
       'пельмень сочнов классический 900г',
       'пельмень сибирский коллекция сочный 800г',
       'пельмень кузнецов царский яство 800г',
       'пельмень мираторга сливочный 800г',
       'пельмень с говядина и свинина', 'пельмень микро с индейка',
       'пельмень микро с говядина', 'пельмень «азиатский»'], dtype=object)

Again - seems decent - and that's it for our toy example of product labelling.